In [1]:
import sys
import os

# Get the parent directory of the current directory
parent_dir = os.path.dirname(os.getcwd())

# Add the parent directory to sys.path
sys.path.append(parent_dir)

#Now you can import a module from the parent directory
from agents import *

In [2]:
########################################
# Agent Class - Fire brigade Agent
########################################
class Fireman2D(Agent):
    location = [0,1]
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def extinguish(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Fire):
            return True
        return False
    
    def grabWater(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False
        
def program(percepts):
    '''Returns an action based on it's percepts'''
    for p in percepts:
        if isinstance(p, Fire):
            return 'extinguish'
        elif isinstance(p, Water):
            return 'grabWater'
        
        if isinstance(p,Bump): # then check if you are at an edge and have to turn
            turn = False
            choice = random.choice((1,2));
        else:
            choice = random.choice((1,3)) # 1-right, 2-left, others-forward
    
        if choice == 1:
            return 'turnright'
        elif choice == 2:
            return 'turnleft'
        else:
            return 'moveforward'

In [3]:
########################################
# THING CLASSES
########################################
# Thing Class - Water
class Water(Thing):
    pass

# Thing Class - Fire
class Fire(Thing):
    pass

In [4]:
from random import choice

########################################
# ENVIRONMENT CLASS
########################################
# Forrest Class
class Forrest2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if not self.is_inbounds(loc):
            things.append(Bump())
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "extinguish":
            items = self.list_things_at(agent.location, tclass=Fire)
            if len(items) != 0:
                if agent.extinguish(items[0]):
                    print('{} estinguish {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        elif action == "grabWater":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.grabWater(items[0]):
                    print('{} grabWater {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more Fire or water'''
        no_edibles = not any(isinstance(thing, Fire) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

In [5]:
collections
collections.Iterable = collections.abc.Iterable
collections.Sequence = collections.abc.Sequence

In [6]:
forrest2D = Forrest2D(20,20, color={'Fireman2D': (200,0,0), 'Water': (0, 200, 200), 'Fire': (230, 115, 40)})

fireman2D = Fireman2D(program)
fire = Fire()
water = Water()
#barbeque = Barbeque()

forrest2D.add_thing(fireman2D,[1,1])
forrest2D.add_thing(water,[0,2])
forrest2D.add_thing(fire,[2,2])

forrest2D.run(200)

,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,


Fireman2D decided to turnright at location: [4, 4]


KeyboardInterrupt: 